In [6]:
import pandas as pd
import nltk.sentiment.sentiment_analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

reviews = pd.read_excel('Edmunds Reviews.xlsx')

reviews = reviews.drop_duplicates()

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
reviews['Review'] = reviews['Review'].str.lower()
reviews["Tokens"] = reviews["Review"].dropna().map(tokenizer.tokenize)

models = ['es','ls','rx','a8','a6','3series','5series','7series','xj','sclass']
model_set = set(models)
reviews['Intersection'] = reviews['Tokens'].map(lambda x: set(x))

reviews['Intersection'] = reviews.Intersection.map(lambda x: set.intersection(x,model_set))
reviews['Intersection'] = reviews['Intersection'].map(lambda x: list(x))

mask = reviews['Intersection'].str.len() != 0
has_models = pd.DataFrame(reviews[mask])

import re
ten_words = []
for index, row in has_models.iterrows():
    for i in row['Intersection']:
        position = row['Tokens'].index(i) 
        mod_sent = row['Tokens'][(position-5):(position+6)]
        ten_words.append(mod_sent)
		
models_sentences = []
for i in ten_words:
    models_sentences.append(" ".join(i))

has_models['Decagrams'] = pd.Series(models_sentences)

mod_sents = []
analyzer = SentimentIntensityAnalyzer()
for sentence in has_models['Decagrams']:
    vs = analyzer.polarity_scores(sentence)
    mod_sents.append(str(vs))

# When you add it to the dataframe it doesn't match up correctly for me, can someone troubleshoot this?
has_models['Model Sentiments'] = mod_sents
